In [2]:
import pdb
import bugzilla
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

import pandas as pd

In [3]:
# tqdm extensions for pandas functions
tqdm.pandas()

## Get All Linked and Associated Bugs

In [4]:
# get the red hat dashboard names
response = requests.get(
    "https://testgrid.k8s.io/redhat-openshift-informing?id=dashboard-group-bar"
)
html = BeautifulSoup(response.content)
testgrid_script = html.findAll("script")[3]
testgrid_script = testgrid_script.text.split()[5].split(",")
dashboard_names = [x.split(":")[1] for x in testgrid_script if "name" in x]
dashboard_names

['"redhat-assisted-installer"',
 '"redhat-openshift-informing"',
 '"redhat-openshift-ocp-release-4.1-blocking"',
 '"redhat-openshift-ocp-release-4.1-informing"',
 '"redhat-openshift-ocp-release-4.2-blocking"',
 '"redhat-openshift-ocp-release-4.2-informing"',
 '"redhat-openshift-ocp-release-4.3-blocking"',
 '"redhat-openshift-ocp-release-4.3-broken"',
 '"redhat-openshift-ocp-release-4.3-informing"',
 '"redhat-openshift-ocp-release-4.4-blocking"',
 '"redhat-openshift-ocp-release-4.4-broken"',
 '"redhat-openshift-ocp-release-4.4-informing"',
 '"redhat-openshift-ocp-release-4.5-blocking"',
 '"redhat-openshift-ocp-release-4.5-broken"',
 '"redhat-openshift-ocp-release-4.5-informing"',
 '"redhat-openshift-ocp-release-4.6-blocking"',
 '"redhat-openshift-ocp-release-4.6-broken"',
 '"redhat-openshift-ocp-release-4.6-informing"',
 '"redhat-openshift-ocp-release-4.7-blocking"',
 '"redhat-openshift-ocp-release-4.7-broken"',
 '"redhat-openshift-ocp-release-4.7-informing"',
 '"redhat-openshift-ocp-re

In [8]:
# bugs linked at timestamps up to this amount of time before today will be returned
max_age = '336h'

# ci details search url
url = 'https://search.ci.openshift.org/'

In [28]:
# get all linked bugs
associated_bugs = list()
all_linked_bugs = set()
for dashboard in tqdm(dashboard_names):
    # get all jobs in this dashboard
    response = requests.get(f"https://testgrid.k8s.io/{dashboard}/summary")
    job_names = response.json().keys()
    
    for job in job_names:
        
        # get all tests in this job
        response = requests.get(f"https://testgrid.k8s.io/{dashboard}/table?&show-stale-tests=&tab={job}")
        
        # params to send to openshift ci search for tests under this job
        args = {
            'type': 'bug+junit',
            'context': '-1',
            'name': job,
            'maxAge': max_age,
        }
        
        for test in response.json().get('tests', []):
            testname = test['name'].split('.', maxsplit=1)[-1]
            
            # use test name as the search phrase
            args['search'] = testname.replace('[', '\[').replace(']', '\]')

            # search for linked and associated bugs for this test
            response = requests.post(url, data=args)
            soup = BeautifulSoup(response.content)

            # the "em" objects in soup have information that can tell us
            # whether or not this test had a linked bug for the given job name
            em_objects = soup.find_all('em')
            for em in em_objects:
                if 'Found' in em.text:
                    pct_affected = float(em.text.split()[2][:-1])

            # init to empty for this test result / reset to empty from previous test result
            test_bugs = []
            
            # if percent jobs affected is 0 then the linked bugs correspond to another job
            if pct_affected > 0:
                result_rows = soup.find('table').find('tbody').find_all('tr')
                for row in result_rows:
                    column_values = row.find_all('td')
                    
                    # if there is only 1 column then the result is a junit, not bug
                    if len(column_values) > 1:
                        # check the second column to make sure it is a bug and not junit details result
                        if column_values[1].text == 'bug':
                            test_bugs.append(column_values[0].text[1:])
                            all_linked_bugs.add(column_values[0].text[1:])

            associated_bugs.append((dashboard, job, test, test_bugs))

 20%|██        | 8/40 [1:19:05<5:16:23, 593.24s/it]


ConnectionError: HTTPSConnectionPool(host='search.ci.openshift.org', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2a36e8ad30>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [29]:
linked_and_associated_bugs = pd.DataFrame(associated_bugs, columns=['dashboard', 'job', 'test_name', 'bug_ids'])
linked_and_associated_bugs.head()

,dashboard,job,test_name,bug_ids
0,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,{'name': 'openshift-tests.[sig-node] pods shou...,"[1916880, 1933760, 1887509, 1943334, 1886920]"
1,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,{'name': 'openshift-tests.[sig-arch] Monitor c...,"[1913751, 1918680]"
2,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,{'name': 'openshift-tests.[sig-network] pods s...,"[1932811, 1882750, 1921720, 1886922, 1887509, ..."
3,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,{'name': 'openshift-tests.[sig-network] pods s...,"[1908378, 1886922, 1882750, 1943334, 1929674, ..."
4,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,{'name': 'openshift-tests.[sig-network] pods s...,"[1908378, 1886922, 1882750, 1932811, 1929674, ..."


## Get Bugzilla Details

In [31]:
bug_details_to_get = [
    'status', 
    'component',
    'components', 
    'platform',
    'priority',
    'product',
    'sub_component',
    'sub_components',
    'summary',
    'tags',
    'target_milestone',
    'target_release',
]

In [34]:
bugs_df = pd.DataFrame(columns=['bug_id'] + bug_details_to_get, index=range(len(all_linked_bugs)))
bugs_df = bugs_df.assign(bug_id=all_linked_bugs)
bugs_df.head()

,bug_id,status,component,components,platform,priority,product,sub_component,sub_components,summary,tags,target_milestone,target_release
0,1883345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1879253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1789596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1772881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1866399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
bzapi = bugzilla.Bugzilla("bugzilla.redhat.com")

def fill_bug_details(bug_row):
    global bzapi
    try:
        bug = bzapi.getbug(bug_row.bug_id)
    except Exception:
        print(f'unauthorized to access {bug_row.bug_id}')
        return bug_row
    for detail in bug_row.index:
        bug_row[detail] = getattr(bug, detail)
    return bug_row

bugs_df.iloc[:100].progress_apply(fill_bug_details, axis=1)
bugs_df

 15%|█▌        | 15/100 [00:09<00:57,  1.48it/s]

unauthorized to access 1888192


 22%|██▏       | 22/100 [00:13<00:45,  1.70it/s]

unauthorized to access 1857084


 23%|██▎       | 23/100 [00:14<00:42,  1.80it/s]

...

unauthorized to access 1857558


 99%|█████████▉| 99/100 [01:03<00:00,  1.55it/s]

unauthorized to access 1881577


100%|██████████| 100/100 [01:05<00:00,  1.53it/s]


,bug_id,status,component,components,platform,priority,product,sub_component,sub_components,summary,tags,target_milestone,target_release
0,1883345,CLOSED,ibm-roks-toolkit,[ibm-roks-toolkit],Unspecified,low,OpenShift Container Platform,,{},Re-enable skipped conformance tests for IBM Cloud,[],---,[4.5.z]
1,1879253,CLOSED,openshift-controller-manager,[openshift-controller-manager],Unspecified,unspecified,OpenShift Container Platform,,{},Add Provider info to synced proxy CA ConfigMap,[],---,[4.6.0]
2,1789596,ASSIGNED,ISV Operators,[ISV Operators],Unspecified,unspecified,OpenShift Container Platform,,{},[ISV] Install Keycloak Operator,[],---,[---]
3,1772881,CLOSED,Insights Operator,[Insights Operator],All,unspecified,OpenShift Container Platform,,{},[RFE] We require insights to create an alert i...,[],---,[4.7.0]
4,1866399,CLOSED,oc,[oc],Unspecified,low,OpenShift Container Platform,,{},must-gather pod retry interval (10 minutes) do...,[],---,[4.6.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2766,1880845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2767,1915007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2768,1910058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2769,1880603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## todo

combine it into a dataframe s.t. each row has one bugzilla id, and columns are status, component, etc + affected tests/jobs.